In [5]:
import xarray as xr
import numpy as np
import pandas as pd

# Example: creating an xarray Dataset with some float data
timestamps = pd.date_range('2024-01-01', periods=500, freq='H')  # 500 timestamps
data = np.random.randn(500)  # 500 random float data values

ds = xr.Dataset({
    'data': ('time', data)
}, coords={'time': timestamps})

ds

C:\Users\lukas\AppData\Local\Temp\ipykernel_3164\3708881943.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range('2024-01-01', periods=500, freq='H')  # 500 timestamps


<xarray.Dataset> Size: 8kB
Dimensions:  (time: 500)
Coordinates:
  * time     (time) datetime64[ns] 4kB 2024-01-01 ... 2024-01-21T19:00:00
Data variables:
    data     (time) float64 4kB -0.7879 0.834 1.788 ... 1.341 -0.8964 0.2116

In [7]:
# Step 1: Reshape the data into chunks of 100
num_groups = len(ds.time) // 100  # number of groups (e.g., 500 samples -> 5 groups)

# Reshape the data to a new shape with the group dimension
data_reshaped = ds['data'].values.reshape(num_groups, 100)

data_reshaped

array([[-7.87861579e-01,  8.34001777e-01,  1.78799482e+00,
        -5.52487222e-01,  1.04760977e+00,  1.31200926e+00,
        -3.91046871e-01, -2.87026705e-01, -2.50010311e-01,
        -1.01904731e+00, -5.29288145e-01, -4.76169195e-01,
        -6.90852233e-01,  1.04309176e-01,  3.27557475e-01,
        -7.83658005e-02, -1.53120625e+00,  1.01173482e-01,
         9.73640633e-02, -8.94277033e-02, -2.73554860e-01,
        -1.13343983e+00, -3.03025298e+00,  1.88115807e-02,
        -1.31936522e+00, -5.08023220e-01,  6.25368025e-01,
        -9.50023470e-01, -7.55633656e-01, -7.26782775e-01,
        -3.29034820e-01,  1.37630336e+00,  7.75398416e-01,
         1.73988170e+00, -5.23688610e-01,  1.64667593e-01,
        -1.36673688e-01,  9.38081828e-01, -1.47003256e+00,
         4.22232067e-01,  7.96763378e-01, -5.55960814e-01,
        -7.36015246e-01,  1.22852691e-01, -4.42744734e-01,
        -7.04076091e-01,  1.19662701e+00,  1.97898280e+00,
         2.15320754e-01,  7.21599387e-02,  1.28469251e+0

In [8]:
# Step 2: Create a new coordinate for group_id
group_id = np.arange(num_groups)
group_id

array([0, 1, 2, 3, 4])

In [9]:
# Step 3: Create a new xarray Dataset with group_id as a dimension
ds_new = xr.Dataset({
    'data': (('group_id', 'time_within_group'), data_reshaped)
}, coords={'group_id': group_id, 'time_within_group': np.arange(100)})

# Print the reshaped dataset
ds_new

<xarray.Dataset> Size: 5kB
Dimensions:            (group_id: 5, time_within_group: 100)
Coordinates:
  * group_id           (group_id) int64 40B 0 1 2 3 4
  * time_within_group  (time_within_group) int64 800B 0 1 2 3 4 ... 96 97 98 99
Data variables:
    data               (group_id, time_within_group) float64 4kB -0.7879 ... ...